In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])


@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
print(cluster,client)

LocalCluster(49fc0159, 'tcp://127.0.0.1:38343', workers=4, threads=16, memory=24.59 GiB) <Client: 'tcp://127.0.0.1:38343' processes=4 threads=16, memory=24.59 GiB>


distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-nek60w5s', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-z_4l8mgf', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-2u9n6rcw', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-wrlg95ih', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-u4mam7ao', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-fnjab4x2', purgin

In [2]:
print(cluster,client)

LocalCluster(49fc0159, 'tcp://127.0.0.1:38343', workers=4, threads=16, memory=24.59 GiB) <Client: 'tcp://127.0.0.1:38343' processes=4 threads=16, memory=24.59 GiB>


In [3]:
#making_label_set
# 
# crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
# disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
# risk = {'1':'초기','2':'중기','3':'말기'}


# #ensemble_labler
# label_description = {}
# for key, value in disease.items():
#     label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
#     for disease_code in value:
#         for risk_code in risk:
#             label = f'{key}_{disease_code}_{risk_code}'
#             label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'
# #crop_labler
# crop_label_description = {}
# for key, value in disease.items():
#     crop_label_description[f'{key}'] = f'{crop[key]}'
    
# global crop_label_encoder
# crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
# crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


# #disease_labler
# disease_label_description = {}
# for key, value in disease.items():
#     disease_label_description[f'00'] = f'{crop[key]}'
#     for disease_code in value:
#         label = f'{disease_code}'
#         disease_label_description[label] = f'{disease_code}'

# global disease_label_encoder
# disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
# disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


# #risk_labler
# risk_label_description = {}
# for key, value in risk.items():
#     label = f'{key}'
#     risk_label_description[label] = f'{key}'

# global risk_label_encoder
# risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
# risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [70]:
label_description = {
                    "1_00_0" : "딸기", 
                    "2_00_0" : "토마토",
                    "2_a5_2" : "토마토_흰가루병_중기",
                    "3_00_0" : "파프리카",
                    "3_a9_1" : "파프리카_흰가루병_초기",
                    "3_a9_2" : "파프리카_흰가루병_중기",
                    "3_a9_3" : "파프리카_흰가루병_말기",
                    "3_b3_1" : "파프리카_칼슘결핍_초기",
                    "3_b6_1" : "파프리카_다량원소결필(N)_초기",
                    "3_b7_1" : "파프리카_다량원소결필(P)_초기",
                    "3_b8_1" : "파프리카_다량원소결필(K)_초기",
                    "4_00_0" : "오이",
                    "5_00_0" : "고추",
                    "5_a7_2" : "고추_탄저병_중기",
                    "5_b6_1" : "고추_다량원소결필(N)_초기",
                    "5_b7_1" : "고추_다량원소결필(P)_초기",
                    "5_b8_1" : "고추_다량원소결필(K)_초기",
                    "6_00_0" : "시설포도",
                    "6_a11_1" : "시설포도_탄저병_초기",
                    "6_a11_2" : "시설포도_탄저병_중기",
                    "6_a12_1" : "시설포도_노균병_초기",
                    "6_a12_2" : "시설포도_노균병_중기",
                    "6_b4_1" : "시설포도_일소피해_초기",
                    "6_b4_3" : "시설포도_일소피해_말기",
                    "6_b5_1" : "시설포도_축과병_초기"   }


global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

In [71]:
@delayed
def label_encoding(label):
    global label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

def getlable(jsonpath):
    with open(jsonpath, 'r') as f:
        json_file = json.load(f)

    crop = json_file['annotations']['crop']
    disease = json_file['annotations']['disease']
    risk = json_file['annotations']['risk']
    label = f'{crop}_{disease}_{risk}'
    return label
    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path)))
        labelarr = np.append(labelarr,label)
    return labelarr

## 이미지셋만 사용 ##

In [5]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [6]:
# row_img = row_img.rechunk(1000)
from tensorflow.keras.applications.resnet import ResNet50
model_RESNET50 = ResNet50(weights='imagenet')
model_RESNET50.compile()

2022-01-21 15:38:15.787301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-21 15:38:15.824735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/lib64:
2022-01-21 15:38:15.824756: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
202

트레인셋

In [50]:
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return img
#making_img_set
row_img = dask_image.imread.imread(path+"/train/*/*.jpg")
train = [imageresize(img) for img in row_img]

minibatch_size = 200
data_length = len(train)
loop_num = int(np.floor(data_length/minibatch_size))
train_x = dask.array.zeros((data_length,1000))

count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        train_x[start_p:end_p] = model_RESNET50(np.array(compute(train[start_p:end_p])[0])).numpy()
        
    else:
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        train_x[start_p:] = model_RESNET50(np.array(compute(train[start_p:])[0])).numpy()
            
    count += 1
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [72]:
label_list = sorted(glob(path+"/train/*/*.json"))
y_train = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = da.array(results)
# print(label.shape)

Exception ignored in: <function ScopedTFGraph.__del__ at 0x7f4d22e31ef0>
Traceback (most recent call last):
  File "/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/tensorflow/python/framework/c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
AttributeError: deleter


테스트셋

In [ ]:
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return img
#making_img_set
row_img = dask_image.imread.imread(path+"/test/*/*.jpg")
test = [imageresize(img) for img in row_img]

minibatch_size = 200
data_length = len(test)
loop_num = int(np.floor(data_length/minibatch_size))
test_x = dask.array.zeros((data_length,1000))

count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        test_x[start_p:end_p] = model_RESNET50(np.array(compute(test[start_p:end_p])[0])).numpy()
        
    else:
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        test_x[start_p:] = model_RESNET50(np.array(compute(test[start_p:])[0])).numpy()
            
    count += 1
    


계층화 스플릿

In [82]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)

In [83]:
XGB = xgb.XGBClassifier(params,eval_metric='mlogloss',use_label_encoder=False)
XGB.fit(X, y)

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, ...)

In [84]:
y_pred = XGB.predict(X_test)

In [85]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[114   0   1  29   0   0   0   0   1   0   1   7   0   0   0   0   0   9
    0   0   0   0   0   0   0]
 [  1  17   0   3   0   0   0   0   0   0   0   5   0   0   0   0   0   3
    0   0   0   0   0   0   0]
 [  1   0  23   3   0   0   0   0   0   0   0   4   0   0   0   2   0   5
    0   0   0   0   0   0   0]
 [ 10   0   1 194   2   1   0   3   1   0   0   5   1   4   3   0   7   3
    0   0   0   0   0   0   0]
 [  2   0   0  14   3   4   0   4   0   1   0   1   0   0   2   0   0   0
    0   0   0   0   0   0   0]
 [  2   0   0   9   2   3   0   2   1   0   0   2   0   0   0   0   0   1
    0   0   0   0   0   0   0]
 [  0   0   1   1   1   0   0   3   0   0   0   1   0   0   0   0   0   1
    0   0   0   0   0   0   0]
 [  1   0   0  14   0   0   0  10   1   1   3   0   0   0   1   0   0   2
    0   0   0   0   0   0   0]
 [  3   0   0  11   0   1   0   0   0   3   5   1   0   0   0   1   2   1
    0   0   0   0   0   0   0]
 [  5   1   0   4   1   0   0   2   0  13   3   0   0  

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [86]:
answer = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_test])
predss = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추,고추_다량원소결필(K)_초기,고추_다량원소결필(N)_초기,고추_다량원소결필(P)_초기,고추_탄저병_중기,딸기,시설포도,시설포도_노균병_중기,시설포도_일소피해_말기,시설포도_탄저병_초기,오이,토마토,토마토_흰가루병_중기,파프리카,파프리카_다량원소결필(K)_초기,파프리카_다량원소결필(N)_초기,파프리카_다량원소결필(P)_초기,파프리카_칼슘결핍_초기,파프리카_흰가루병_중기,파프리카_흰가루병_초기
answer,,,,,,,,,,,,,,,,,,,,
고추,0,0,0,0,0,0,2,0,0,1,0,0,0,10,0,1,0,0,0,0
고추_다량원소결필(K)_초기,0,13,4,0,0,1,0,0,0,0,0,1,0,6,3,0,0,2,0,1
고추_다량원소결필(N)_초기,0,5,15,0,0,1,0,0,0,0,0,0,0,3,1,0,0,5,0,0
고추_다량원소결필(P)_초기,0,0,0,19,1,1,0,0,0,0,0,0,1,10,0,0,0,0,0,0
고추_탄저병_중기,0,0,0,0,15,0,1,0,0,0,0,0,0,4,0,0,0,0,0,0
딸기,0,0,0,0,0,114,9,0,0,0,7,0,1,29,1,1,0,0,0,0
시설포도,0,0,0,0,1,10,126,0,0,1,17,1,2,6,0,0,1,1,0,0
시설포도_노균병_중기,0,0,0,0,0,0,4,1,0,0,1,0,0,0,0,0,0,0,0,0
시설포도_노균병_초기,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0


In [87]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.36295844665985966

## csv만 사용 ##

트레인셋

In [88]:
#making_csv_set
row_csv = dd.read_csv(path+"/train/*/*.csv",dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
       '내부 이슬점 최저': 'object',
       '내부 이슬점 평균': 'object'})
partitions = row_csv.to_delayed()
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
X_train = np.array(dask.compute(*datas))


In [89]:
label_list = sorted(glob(path+"/train/*/*.json"))
y_train = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)

In [90]:
label.shape, X_train.shape

((5767,), (5767, 290, 9))

In [91]:
X, X_test, y, y_test = train_test_split(X_train, label, test_size=0.2, shuffle= True,stratify=label)

In [92]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

for i in range(len(csv_features)):
  start = time.time()
  globals()["XGB_{}".format(i)] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X[:,:,i],y)
  print(time.time()-start)

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


8.76465129852295
6.4558022022247314
5.716325044631958
10.368138313293457
9.497236728668213
8.114715814590454
5.529173135757446
6.151880979537964
6.860687732696533


In [93]:
x0=XGB_0.predict(X_train[:,:,0])
x1=XGB_1.predict(X_train[:,:,1])
x2=XGB_2.predict(X_train[:,:,2])
x3=XGB_3.predict(X_train[:,:,3])
x4=XGB_4.predict(X_train[:,:,4])
x5=XGB_5.predict(X_train[:,:,5])
x6=XGB_6.predict(X_train[:,:,6])
x7=XGB_7.predict(X_train[:,:,7])
x8=XGB_8.predict(X_train[:,:,8])
processed_x = np.array([x0,x1,x2,x3,x4,x5,x6,x7,x8,]).T

In [94]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_G = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(processed_x,label)
print(time.time()-start)

2.6289143562316895


In [99]:
x0_val=XGB_0.predict(X_test[:,:,0])
x1_val=XGB_1.predict(X_test[:,:,1])
x2_val=XGB_2.predict(X_test[:,:,2])
x3_val=XGB_3.predict(X_test[:,:,3])
x4_val=XGB_4.predict(X_test[:,:,4])
x5_val=XGB_5.predict(X_test[:,:,5])
x6_val=XGB_6.predict(X_test[:,:,6])
x7_val=XGB_7.predict(X_test[:,:,7])
x8_val=XGB_8.predict(X_test[:,:,8])
processed_x_val = np.array([x0_val,x1_val,x2_val,x3_val,x4_val,x5_val,x6_val,x7_val,x8_val,]).T

In [100]:
y_pred = XGB_G.predict(processed_x_val)

In [101]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[162   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0  29   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 235   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0  24   7   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0  10  12   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   2   6   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  30   0   0   0   0   0   0   3   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  24   0   3   0   0   0   1   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  31   0   0   0  

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [102]:
answer = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_test])
predss = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추,고추_다량원소결필(K)_초기,고추_다량원소결필(N)_초기,고추_다량원소결필(P)_초기,고추_탄저병_중기,딸기,시설포도,시설포도_노균병_중기,시설포도_노균병_초기,시설포도_일소피해_말기,...,오이,토마토,토마토_흰가루병_중기,파프리카,파프리카_다량원소결필(K)_초기,파프리카_다량원소결필(N)_초기,파프리카_다량원소결필(P)_초기,파프리카_칼슘결핍_초기,파프리카_흰가루병_중기,파프리카_흰가루병_초기
answer,,,,,,,,,,,,,,,,,,,,,
고추,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_다량원소결필(K)_초기,0,29,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
고추_다량원소결필(N)_초기,0,0,28,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
고추_다량원소결필(P)_초기,0,0,0,32,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_탄저병_중기,0,0,0,0,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
딸기,0,0,0,0,0,162,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도,0,0,0,0,0,0,166,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_노균병_중기,0,0,0,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_노균병_초기,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.9079928444953416